## Requirements
* python >= 3.7
* [jq](https://stedolan.github.io/jq/)

In [ ]:
! apt install -y jq

## Clone repo and init env

In [ ]:
%%bash
[ ! -d starwhale ] && git clone https://github.com/star-whale/starwhale
cd starwhale/example/mnist/
git lfs pull
pip install --upgrade pip && pip install starwhale -U

## Run Mnist demo

In [ ]:
%cd starwhale/example/mnist/

In [ ]:
%%bash
ls -lha
swcli runtime quickstart shell . --python-env=venv --create-env --name pytorch-mnist --force
. .venv/bin/activate
python3 -m pip install -r requirements-sw-lock.txt  --pre -U

In [ ]:
%%bash
. .venv/bin/activate
swcli instance select local
swcli runtime build . && swcli runtime list

In [ ]:
! . .venv/bin/activate; swcli model build . && swcli model list

In [ ]:
%%bash

mkdir -p data && pushd data
[[ ! -f train-images-idx3-ubyte ]] && wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
[[ ! -f train-labels-idx1-ubyte ]] && wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
[[ ! -f t10k-images-idx3-ubyte ]] && wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
[[ ! -f t10k-labels-idx1-ubyte ]] && wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

find . -maxdepth 1 -type f -name "*.gz" 2>/dev/null | xargs -I{} sh -c "echo {} && gzip -d {}"  
popd

In [ ]:
! . .venv/bin/activate; swcli dataset build . && swcli dataset list

In [ ]:
%%bash
. .venv/bin/activate; 
swcli eval run --model mnist/version/latest --dataset mnist/version/latest
swcli eval list

In [ ]:
! . .venv/bin/activate; swcli -o json eval list | jq -r '. | first | .manifest.version'

## Get and show results

In [ ]:
! . .venv/bin/activate; env COLUMNS=150 swcli eval info $(swcli -o json eval list | jq -r '. | first | .manifest.version')

In [ ]:
! pip install matplotlib

import subprocess
import json

cmd = ". .venv/bin/activate; swcli -o json eval info $(swcli -o json eval list | jq -r '. | first | .manifest.version')"
result = subprocess.check_output(cmd, shell=True)
data = json.loads(result)['report']

In [ ]:
! pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
cm = data['confusion_matrix']['binarylabel']
cm = [list(i.values())[:-1] for i in cm]
ax = sns.heatmap(cm, ax=ax, annot=True, cmap='Greens', fmt='.4f')
ax.invert_yaxis()